In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from scipy.stats.kde import gaussian_kde
from numpy import linspace
import time
from scipy import interpolate

In [2]:
from natal_kick_tools import mandel_muller_likelihood_functions as mmf

# Probability from COMPAS simulated models

In [3]:
# Define the models of interest
bh_kicks=[200]
ns_kicks = [400]
sigmas = [0.3, 0.7]

# Define the location of the posterior data
pulsar_data_loc = "../correctedPulsarPosteriors/correctedVtData"

In [ ]:
start = time.time() 
p_pulsar_given_model = mmf.get_pulsar_probability(pulsar_data_loc, bh_kicks=bh_kicks, ns_kicks=ns_kicks, sigmas=sigmas)
end = time.time()

print("Complete calculation completed in:", end - start, "s")

Loading Mandel Muller model data from /work2/08178/vkapil/frontera/supernova_remnant/bh_200_ns_400_sigma_0.3_combined.h5
Loading Mandel Muller model data from /work2/08178/vkapil/frontera/supernova_remnant/bh_200_ns_400_sigma_0.7_combined.h5


In [ ]:
p_pulsar_given_model